In [1]:
# Importing necessary packages

import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import json
import time
import sqlite3

## Pulling & Importing US Events Data

In [2]:
# Setting API Key param
params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni'}

# Pulling URIs for all country codes
uri_response = requests.get('https://app.ticketmaster.com/discovery-feed/v2/events', params=params)

# Extracting US URI
uri_response.json()

{'countries': {'AT': {'CSV': {'compressed_md5_checksum': '7af59d0766383403a602e2dd980413b2',
    'compressed_size_bytes': 31174,
    'country_code': 'AT',
    'format': 'CSV',
    'last_updated': '2018-07-28T17:04:42Z',
    'num_events': 254,
    'uri': 'https://s3.amazonaws.com/dc-feeds-prod1-public/20180728/EVENTS_RAW-AT-1588953e-4b6d-4def-9774-d272b248d00a-2018-07-28_050441.csv.gz'},
   'JSON': {'compressed_md5_checksum': '8db446c90634ff4a6ebf89a4e458f1b4',
    'compressed_size_bytes': 244675,
    'country_code': 'AT',
    'format': 'JSON',
    'last_updated': '2018-07-28T17:04:42Z',
    'num_events': 254,
    'uri': 'https://s3.amazonaws.com/dc-feeds-prod1-public/20180728/EVENTS_RAW-AT-7ce406a6-9a5d-455b-8309-e86edd130533-2018-07-28_050441.json.gz'},
   'XML': {'compressed_md5_checksum': '9d3d1f89c5e23bbe73864f29d0251cbd',
    'compressed_size_bytes': 253133,
    'country_code': 'AT',
    'format': 'XML',
    'last_updated': '2018-07-28T17:04:43Z',
    'num_events': 254,
    'uri':

In [3]:
# Reading US events JSON
tm_events = pd.read_json('/Users/Albert/Desktop/ticketmaster_US_events.json')

# Processing nested JSON in DataFrame
tm_events = json_normalize(tm_events['events'])

# Limiting to music events only
tm_events = tm_events[tm_events['classificationSegment']=='Music']

# Extracting interesting features
tm_events = tm_events[['eventId',
  'eventName',
  'attractions',
  'classificationGenre',
  'classificationGenreId',
  'classificationSubGenre',
  'classificationSubGenreId',
  'venue.venueId',
  'venue.venueName',
  'venue.venueStateCode',
  'venue.venueCity',
  'venue.venueZipCode',  
  'maxPrice',
  'minPrice',
  'onsaleStartDateTime',
  'eventStartDateTime',
  'eventStartLocalDate',
  'eventStartLocalTime',
  'presales',
  'promoters',
  'eventStatus'
 ]]

# Removing events with no artist information
tm_events = tm_events[tm_events['attractions'].astype('bool')]

# Removing cancelled events
tm_events = tm_events[tm_events['eventStatus'] != 'cancelled']

# Removing events with no pricing information
tm_events = tm_events[(tm_events['maxPrice'].astype('bool'))|(tm_events['minPrice'].astype('bool'))]

# Create attribute for max/min price differential
tm_events['maxPrice'] = tm_events['maxPrice'].astype('float')
tm_events['minPrice'] = tm_events['minPrice'].astype('float')
tm_events['max_min_differential'] = tm_events['maxPrice']-tm_events['minPrice']

# Converting presales into Boolean
tm_events['presales'] = np.where((tm_events['presales']), 1, 0)

print(tm_events.shape)
tm_events.head()

(12042, 22)


,eventId,eventName,attractions,classificationGenre,classificationGenreId,classificationSubGenre,classificationSubGenreId,venue.venueId,venue.venueName,venue.venueStateCode,...,maxPrice,minPrice,onsaleStartDateTime,eventStartDateTime,eventStartLocalDate,eventStartLocalTime,presales,promoters,eventStatus,max_min_differential
3,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,[{'attraction': {'attractionUrl': 'https://www...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAEet7A,Ascend Amphitheater part of the Journeys Conce...,TN,...,114.5,29.50,2018-03-30T15:00:00Z,2018-08-12T23:30:00Z,2018-08-12,18:30,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,85.00
10,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZA6tFlA,The Fillmore Silver Spring presented by Cricke...,MD,...,15.5,15.50,2018-06-08T14:00:00Z,2018-08-05T00:00:00Z,2018-08-04,20:00,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,0.00
12,1AyZAC7Gkd9QyRb,Legends In Concert (Las Vegas),[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,...,82.5,52.75,2017-12-05T02:00:00Z,2018-09-28T04:30:00Z,2018-09-27,21:30,0,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,29.75
15,1AyZAC7GkdKbzEd,Legends In Concert (Las Vegas),[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,...,82.5,52.75,2017-12-05T02:00:00Z,2018-07-26T23:00:00Z,2018-07-26,16:00,0,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,29.75
16,G5viZ4A2p8MNN,Metallica - WorldWired Tour,[{'attraction': {'attractionUrl': 'https://www...,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,KovZpa61pe,Verizon Arena,AR,...,135.0,65.00,2018-03-02T16:00:00Z,2019-01-21T01:30:00Z,2019-01-20,19:30,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,70.00


In [4]:
## Create attribute for total upcoming events by venue

# Set for loop to pull 200 values from API at a time
venueIds = tm_events['venue.venueId'].drop_duplicates()
loop_count = tm_events.shape[0]//200
# Create empty DF to store pulled data
venue_upcomingEvents_count = pd.DataFrame()

# Instantiate for loop
for i in range(loop_count):
    # Set pull request range
    if i != loop_count-1:
        id_pull_request = list(venueIds[i*200:(i+1)*200])
    else:
        id_pull_request = list(venueIds[i*200:])
    # Calling API for set of 200 values
    id_pull_request = ','.join(id_pull_request)
    params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
              'id':id_pull_request,
              'size':'200'
             }
    response = requests.get('https://app.ticketmaster.com/discovery/v2/venues.json', params=params)
    # Processing reponse & appending to DF
    response_processed = json_normalize(response.json())['_embedded.venues'][0]
    response_processed = json_normalize(response_processed)
    response_processed = response_processed[['id','upcomingEvents._total','upcomingEvents.ticketmaster']]
    venue_upcomingEvents_count = venue_upcomingEvents_count.append(response_processed)
    # Set time lapse to prevent triggering API pull limits
    time.sleep(0.250)
    

# Merge upcoming venue count with general DF
venue_upcomingEvents_count.columns = ['venue.venueId','venue_totalupcomingEvents','venue_ticketmasterupcomingEvents']
tm_events = tm_events.merge(venue_upcomingEvents_count, how='left', on=['venue.venueId'])
# Drop duplicates
tm_events = tm_events.drop_duplicates(subset='eventId')
# Remove null values
tm_events = tm_events[tm_events['venue_ticketmasterupcomingEvents'].notnull()]
print(tm_events.shape)
tm_events.head(10)

(12029, 24)


,eventId,eventName,attractions,classificationGenre,classificationGenreId,classificationSubGenre,classificationSubGenreId,venue.venueId,venue.venueName,venue.venueStateCode,...,onsaleStartDateTime,eventStartDateTime,eventStartLocalDate,eventStartLocalTime,presales,promoters,eventStatus,max_min_differential,venue_totalupcomingEvents,venue_ticketmasterupcomingEvents
0,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,[{'attraction': {'attractionUrl': 'https://www...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAEet7A,Ascend Amphitheater part of the Journeys Conce...,TN,...,2018-03-30T15:00:00Z,2018-08-12T23:30:00Z,2018-08-12,18:30,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,85.00,20,19.0
55,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZA6tFlA,The Fillmore Silver Spring presented by Cricke...,MD,...,2018-06-08T14:00:00Z,2018-08-05T00:00:00Z,2018-08-04,20:00,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,0.00,50,50.0
56,1AyZAC7Gkd9QyRb,Legends In Concert (Las Vegas),[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,...,2017-12-05T02:00:00Z,2018-09-28T04:30:00Z,2018-09-27,21:30,0,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,29.75,261,261.0
57,1AyZAC7GkdKbzEd,Legends In Concert (Las Vegas),[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,...,2017-12-05T02:00:00Z,2018-07-26T23:00:00Z,2018-07-26,16:00,0,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,29.75,261,261.0
58,G5viZ4A2p8MNN,Metallica - WorldWired Tour,[{'attraction': {'attractionUrl': 'https://www...,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,KovZpa61pe,Verizon Arena,AR,...,2018-03-02T16:00:00Z,2019-01-21T01:30:00Z,2019-01-20,19:30,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,70.00,24,18.0
59,vvG1iZ46VboPpY,Hunter Hayes - Runaway June - Coffey Anderson,[{'attraction': {'attractionUrl': 'https://www...,Country,KnvZfZ7vAv6,Country,KZazBEonSMnZfZ7vAFa,KovZpZAF6EaA,The Pacific Amphitheatre,CA,...,2018-04-14T17:00:00Z,2018-08-02T02:00:00Z,2018-08-01,19:00,1,[],onsale,43.00,23,21.0
95,17C8vfG65kLZeeC,The Bronx Wanderers,[{'attraction': {'attractionUrl': 'https://www...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAJalFA,Windows at Bally's Las Vegas,NV,...,2018-05-21T17:00:00Z,2018-08-31T03:00:00Z,2018-08-30,20:00,0,[],onsale,24.31,503,503.0
96,vvG1OZ4GSCsfP5,CloZee Evasion Tour with Special Guests,[{'attraction': {'attractionUrl': 'https://www...,Dance/Electronic,KnvZfZ7vAvF,Club Dance,KZazBEonSMnZfZ7vAJ1,KovZpZA77JtA,The Shelter,MI,...,2018-06-29T13:00:00Z,2018-10-04T02:00:00Z,2018-10-03,22:00,1,"[{'promoter': {'id': '653', 'name': 'LIVE NATI...",onsale,0.00,23,23.0
151,vv1AaZA8MGkdobIGv,Gino Vannelli,[{'attraction': {'attractionUrl': 'https://www...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAJtF6A,The Libbey Bowl,CA,...,2018-01-31T18:00:00Z,2018-08-05T02:00:00Z,2018-08-04,19:00,1,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,0.00,8,8.0
152,G5vVZ4dxwJee3,Legends In Concert Holiday Show,[{'attraction': {'attractionUrl': 'https://www...,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAEkFlA,The Fox Theater at Foxwoods Resort Casino,CT,...,2018-01-26T15:00:00Z,2018-12-29T01:00:00Z,2018-12-28,20:00,0,"[{'promoter': {'id': '494', 'name': 'PROMOTED ...",onsale,10.00,68,68.0


In [6]:
## Extracting feature info from 'attractions' nested JSON

# Creating empty DF to store attractions data
attractions = pd.DataFrame()

# Creating multi-index DF to store all attraction IDs for further feature generation
Index = pd.MultiIndex(levels=[[],[]],
                      labels=[[],[]],
                      names=[u'event_idx', u'artist_idx']
                     )
Columns = [u'id']
attractionIds = pd.DataFrame(index=Index, columns=Columns)


# Extracting artist information row by row
for i in tm_events.index:
    observation = json_normalize(tm_events.loc[i, 'attractions'])

    # Create attribute for count of acts
    attraction_count = observation.shape[0]
    observation['attractionCount'] = attraction_count

    
    # Create attribute for multiple genres & subGenres
    multigenres = False
    multisubGenres = False

    if attraction_count > 1:
        for j in range(1, attraction_count):
            if observation.loc[0, 'attraction.classificationGenre'] != observation.loc[j, 'attraction.classificationGenre']:
                multigenres = True
            if observation.loc[0, 'attraction.classificationSubGenre'] != observation.loc[j, 'attraction.classificationSubGenre']:
                multisubGenres = True
    
    observation['multiGenres'] = multigenres
    observation['multiSubGenres'] = multisubGenres
    
    
    # Add artist ids to empty DF
    for j in observation.index:
        attractionIds.loc[(i, j), :] = observation.loc[j, 'attraction.attractionId']
    
    
    # Keeping relevant attributes
    observation = observation[['attraction.attractionId',
                                 'attraction.attractionName',
                                 'attractionCount',
                                 'attraction.classificationGenre',
                                 'attraction.classificationGenreId',
                                 'multiGenres',
                                 'attraction.classificationSubGenre',
                                 'attraction.classificationSubGenreId',
                                 'multiSubGenres'
                                ]]
    attractions = attractions.append(observation.iloc[0,:])

    
attractions.index = tm_events.index
tm_events = pd.concat([tm_events, attractions], axis=1)
del tm_events['attractions']

print(tm_events.shape)
tm_events.head()

(12029, 32)


,eventId,eventName,classificationGenre,classificationGenreId,classificationSubGenre,classificationSubGenreId,venue.venueId,venue.venueName,venue.venueStateCode,venue.venueCity,...,venue_ticketmasterupcomingEvents,attraction.attractionId,attraction.attractionName,attraction.classificationGenre,attraction.classificationGenreId,attraction.classificationSubGenre,attraction.classificationSubGenreId,attractionCount,multiGenres,multiSubGenres
0,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAEet7A,Ascend Amphitheater part of the Journeys Conce...,TN,Nashville,...,19.0,K8vZ91711TV,Lucinda Williams,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,4.0,1.0,1.0
55,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZA6tFlA,The Fillmore Silver Spring presented by Cricke...,MD,Silver Spring,...,50.0,K8vZ917uMC0,Strangelove,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,2.0,0.0,0.0
56,1AyZAC7Gkd9QyRb,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,Las Vegas,...,261.0,K8vZ917KQ77,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,1.0,0.0,0.0
57,1AyZAC7GkdKbzEd,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,Las Vegas,...,261.0,K8vZ917KQ77,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,1.0,0.0,0.0
58,G5viZ4A2p8MNN,Metallica - WorldWired Tour,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,KovZpa61pe,Verizon Arena,AR,North Little Rock,...,18.0,K8vZ9171G9V,Metallica,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,1.0,0.0,0.0


In [7]:
## Create attribute for summed upcoming events by artist

# Isolate attraction IDs for API call
attractionIds_series = attractionIds['id'].drop_duplicates()

# Set for loop iterations
loop_count = attractionIds.shape[0]//200

# Create empty DF to store pulled data
attraction_upcomingEvents_count = pd.DataFrame()

# Instantiate for loop
for i in range(loop_count):
    # Set pull request range
    if i != loop_count-1:
        id_pull_request = list(attractionIds_series[i*200:(i+1)*200])
    elif i == loop_count-1:
        id_pull_request = list(attractionIds_series[i*200:])
    # Calling API for set of 200 values
    id_pull_request = ','.join(id_pull_request)
    params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
              'id':id_pull_request,
              'size':'200'
             }
    response = requests.get('https://app.ticketmaster.com/discovery/v2/attractions.json', params=params)
    # Processing reponse & appending to DF
    response_processed = json_normalize(response.json())['_embedded.attractions'][0]
    response_processed = json_normalize(response_processed)
    response_processed = response_processed[['id','upcomingEvents._total','upcomingEvents.ticketmaster']]
    attraction_upcomingEvents_count = attraction_upcomingEvents_count.append(response_processed)
    # Set time lapse to prevent triggering API pull limits
    print(i)
    time.sleep(0.250)

    
# Drop artist ID duplicates for join
attraction_upcomingEvents_count = attraction_upcomingEvents_count.drop_duplicates()

# Merge and sort event count with artist ID multi-index
attractionIds = pd.merge(attractionIds.reset_index(), attraction_upcomingEvents_count, on=['id']).set_index(attractionIds.index.names)
attractionIds = attractionIds.sortlevel()

# Group mean of upcoming events for all artists by event and adjust column names
attractionIds = attractionIds.groupby(level=0).mean()
attractionIds.columns = ['artist_totalupcomingEvents', 'artist_ticktmasterupcomingEvents']

# Add feature set to original dataset
tm_events = tm_events.join(attractionIds, how='inner')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


/Users/Albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)


In [8]:
## Extracting promoter name
    
# Extracting promoter information row by row
for i in tm_events.index:
    # If no promoter info listed return value 'Not Listed'
    if not tm_events['promoters'].astype('bool')[i]:
        tm_events.loc[i, 'promoters'] = 'NOT LISTED'
    # Else return name of promoter
    else:
        tm_events.loc[i, 'promoters'] = json_normalize(tm_events.loc[i, 'promoters'][0])['promoter.name'][0]

In [9]:
# Final column name adjustments to remove '.'
rename_dict = {'venue.venudId':'venueId',
               'venue.venueName':'venueName',
               'venue.venueStateCode':'venueStateCode',
               'venue.venueCity':'venueCity',
               'venue.venueZipCode':'venueZipCode',
               'attraction.attractionId':'attractionId',
               'attraction.attractionName':'attractionName',
               'attraction.classificationGenre':'attractionGenre',
               'attraction.classificationGenreId':'attractionGenreId',
               'attraction.classificationSubGenre':'attractionSubGenre',
               'attraction.classificationSubGenreId':'attractionSubGenreId'
              }

tm_events.rename(columns=rename_dict, inplace=True)


# Dump dataset into SQL DB
conn = sqlite3.connect('Ticket Information.db')
tm_events.to_sql('ticketmaster', conn, if_exists='replace')
conn.close()

# Print dataset details
print (tm_events.shape)
print (tm_events.columns)
tm_events.head(10)

(11982, 34)
Index(['eventId', 'eventName', 'classificationGenre', 'classificationGenreId',
       'classificationSubGenre', 'classificationSubGenreId', 'venue.venueId',
       'venueName', 'venueStateCode', 'venueCity', 'venueZipCode', 'maxPrice',
       'minPrice', 'onsaleStartDateTime', 'eventStartDateTime',
       'eventStartLocalDate', 'eventStartLocalTime', 'presales', 'promoters',
       'eventStatus', 'max_min_differential', 'venue_totalupcomingEvents',
       'venue_ticketmasterupcomingEvents', 'attractionId', 'attractionName',
       'attractionGenre', 'attractionGenreId', 'attractionSubGenre',
       'attractionSubGenreId', 'attractionCount', 'multiGenres',
       'multiSubGenres', 'artist_totalupcomingEvents',
       'artist_ticktmasterupcomingEvents'],
      dtype='object')


,eventId,eventName,classificationGenre,classificationGenreId,classificationSubGenre,classificationSubGenreId,venue.venueId,venueName,venueStateCode,venueCity,...,attractionName,attractionGenre,attractionGenreId,attractionSubGenre,attractionSubGenreId,attractionCount,multiGenres,multiSubGenres,artist_totalupcomingEvents,artist_ticktmasterupcomingEvents
0,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAEet7A,Ascend Amphitheater part of the Journeys Conce...,TN,Nashville,...,Lucinda Williams,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,4.0,1.0,1.0,19.75,17.25
55,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZA6tFlA,The Fillmore Silver Spring presented by Cricke...,MD,Silver Spring,...,Strangelove,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,2.0,0.0,0.0,6.50,5.50
56,1AyZAC7Gkd9QyRb,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,Las Vegas,...,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,1.0,0.0,0.0,201.00,201.00
57,1AyZAC7GkdKbzEd,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAJanIA,Donny & Marie Showroom at Flamingo Las Vegas,NV,Las Vegas,...,Legends In Concert (Las Vegas),Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,1.0,0.0,0.0,201.00,201.00
58,G5viZ4A2p8MNN,Metallica - WorldWired Tour,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,KovZpa61pe,Verizon Arena,AR,North Little Rock,...,Metallica,Metal,KnvZfZ7vAvt,Heavy Metal,KZazBEonSMnZfZ7vkFd,1.0,0.0,0.0,46.00,37.00
59,vvG1iZ46VboPpY,Hunter Hayes - Runaway June - Coffey Anderson,Country,KnvZfZ7vAv6,Country,KZazBEonSMnZfZ7vAFa,KovZpZAF6EaA,The Pacific Amphitheatre,CA,Costa Mesa,...,Hunter Hayes,Country,KnvZfZ7vAv6,Country,KZazBEonSMnZfZ7vAFa,2.0,0.0,0.0,8.50,8.00
95,17C8vfG65kLZeeC,The Bronx Wanderers,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAJalFA,Windows at Bally's Las Vegas,NV,Las Vegas,...,The Bronx Wanderers,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,1.0,0.0,0.0,155.00,154.00
96,vvG1OZ4GSCsfP5,CloZee Evasion Tour with Special Guests,Dance/Electronic,KnvZfZ7vAvF,Club Dance,KZazBEonSMnZfZ7vAJ1,KovZpZA77JtA,The Shelter,MI,Detroit,...,Clozee,Dance/Electronic,KnvZfZ7vAvF,Dance/Electronic,KZazBEonSMnZfZ7vA1E,2.0,0.0,1.0,15.00,5.00
151,vv1AaZA8MGkdobIGv,Gino Vannelli,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,KovZpZAJtF6A,The Libbey Bowl,CA,Ojai,...,Gino Vannelli,Rock,KnvZfZ7vAeA,Pop,KZazBEonSMnZfZ7v6F1,1.0,0.0,0.0,8.00,3.00
152,G5vVZ4dxwJee3,Legends In Concert Holiday Show,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,KovZpZAEkFlA,The Fox Theater at Foxwoods Resort Casino,CT,Mashantucket,...,Legends In Concert,Undefined,KnvZfZ7vAe6,Undefined,KZazBEonSMnZfZ7v6JI,1.0,0.0,0.0,112.00,112.00


_________________________

# ARCHIVED

## Testing Ticketmaster API

In [2]:
# Setting API params (US based, music events only)
params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
          'countryCode':'US',                  # US based events only
          'segmentName':'Music',               # Music events only
          'sort':'date,asc',                  # Sorted by descending date
          'size':'200'}                        # API only allows 200 events pulled at a time

# Calling Ticketmaster API
response = requests.get('https://app.ticketmaster.com/discovery/v2/events.json', params=params)

# Converting response to JSON file
response_json = response.json()

In [3]:
# Convert JSON response to DataFrame
response_df = json_normalize(response_json)

print ('Total Events: {}'.format(response_df['page.totalElements'][0]))
print ('Total Pages: {}'.format(response_df['page.totalPages'][0]))

Total Events: 32641
Total Pages: 164


## Feature Engineering

In [4]:
# Flattening nested JSON
events_df = json_normalize(response_df['_embedded.events'][0])

# Dropping null values from attributes of interest (this helps further break out nested JSON)
events_df.dropna(axis=0,
                 inplace=True,
                 subset=[['id',
                          'name',
                          '_embedded.attractions',
                          '_embedded.venues',
                          'classifications',
                          'priceRanges']])

events_df.columns

Index(['_embedded.attractions', '_embedded.venues', '_links.attractions',
       '_links.self.href', '_links.venues', 'accessibility.info',
       'classifications', 'dates.spanMultipleDays', 'dates.start.dateTBA',
       'dates.start.dateTBD', 'dates.start.dateTime', 'dates.start.localDate',
       'dates.start.localTime', 'dates.start.noSpecificTime',
       'dates.start.timeTBA', 'dates.status.code', 'dates.timezone', 'id',
       'images', 'info', 'locale', 'name', 'outlets', 'pleaseNote',
       'priceRanges', 'promoter.description', 'promoter.id', 'promoter.name',
       'promoters', 'sales.presales', 'sales.public.endDateTime',
       'sales.public.startDateTime', 'sales.public.startTBD',
       'seatmap.staticUrl', 'test', 'type', 'url'],
      dtype='object')

In [24]:
# Flattening artists JSON
def process_artists(events_json):
    
    # Creating empty DF
    artists = pd.DataFrame()
    
    # Extracting artist information row by row
    for i in events_json.index:
        observation = json_normalize(events_json.loc[i, '_embedded.attractions'], record_path='classifications',
                                     meta=['id','name'],
                                     errors='ignore'
                                    )

        # Create attribute for summed upcoming events
        observation['artist.upcomingEvents.total'] = json_normalize(events_json.loc[i, '_embedded.attractions'])['upcomingEvents._total'].sum()
        observation['artist.upcomingEvents.ticketmaster'] = json_normalize(events_json.loc[i, '_embedded.attractions'])['upcomingEvents.ticketmaster'].sum()

        # Create attribute for count of acts
        artist_count = observation.shape[0]
        observation['artist.Count'] = artist_count

        # Create attribute for multiple genres & subGenres
        genres = json_normalize(observation['genre'])
        subGenres = json_normalize(observation['subGenre'])

        primary_genre = genres['genre'][0]['name']
        primary_subGenre = subGenres['subGenre'][0]['name']

        multigenres = False
        multisubGenres = False

        if artist_count > 1:
            for j in range(1, artist_count):
                if primary_genre != genres['genre'][j]['name']:
                    multigenres = True
                if primary_subGenre != subGenres['subGenre'][j]['name']:
                    multisubGenres = True
        observation['multi.genres'] = multigenres
        observation['multi.subGenres'] = multisubGenres

        artists = artists.append(observation.iloc[0,:])


    # Upating columns for clarity
    artists.rename(columns={'id':'primary.artistID', 'name':'primary.artistName'}, inplace=True)

    # Keeping valuable features
    artists = artists[['primary.artistID','primary.artistName','artist.Count','multi.genres','multi.subGenres',
                       'artist.upcomingEvents.ticketmaster','artist.upcomingEvents.total']]

    # Updating index for future join
    artists.index = events_json.index

    return artists


artists = process_artists(events_df)
print(artists.shape)
artists.head()

(23, 7)


,primary.artistID,primary.artistName,artist.Count,multi.genres,multi.subGenres,artist.upcomingEvents.ticketmaster,artist.upcomingEvents.total
85,K8vZ91718N0,ZZ Top,1.0,0.0,0.0,15.0,28.0
86,K8vZ91718N0,ZZ Top,2.0,1.0,1.0,16.0,29.0
120,K8vZ9171G-7,Elton John,1.0,0.0,0.0,64.0,107.0
123,K8vZ9171G-7,Elton John,1.0,0.0,0.0,64.0,107.0
133,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0


In [8]:
# Flattening venues JSON
def process_venues(events_json):
    
    # Create empty DF
    venues = pd.DataFrame()

    # Extracting venue information row by row
    for i in events_json.index:
        observation = json_normalize(events_json.loc[i, '_embedded.venues'])
        venues = venues.append(observation)

    # Keeping valuable attributes
    venues = venues[['id','name','country.countryCode','state.name','city.name','upcomingEvents._total','upcomingEvents.ticketmaster']]

    # Updating index for future join
    venues.index = events_json.index
    # Updating column names for clarity
    venues.columns = [['venue.id','venue.name','country','state','city','venue.upcomingEvents.total','venue.upcomingEvents.ticketmaster']]

    return venues

    
venues = process_venues(events_df)
print(venues.shape)
venues.head()

(5, 7)


,venue.id,venue.name,country,state,city,venue.upcomingEvents.total,venue.upcomingEvents.ticketmaster
68,KovZpZAJeFeA,Event Center at San Jose State University,US,California,San Jose,6,6
162,KovZpZAEA7eA,House of Blues Boston presented by Cricket Wir...,US,Massachusetts,Boston,54,54
163,KovZpZAFF1tA,Center Stage Theater,US,Georgia,Atlanta,29,27
166,KovZpZAFF1tA,Center Stage Theater,US,Georgia,Atlanta,29,27
183,KovZpZA6tdaA,Riverbend Music Center,US,Ohio,Cincinnati,23,23


In [9]:
# Flattening classifications JSON
def process_classifications(events_json):
    
    # Create empty DF
    classifications = pd.DataFrame()

    # Extracting classification information row by row
    for i in events_json.index:
        observation = json_normalize(events_json.loc[i, 'classifications'])
        classifications = classifications.append(observation)

    # Including potentially valuabe features
    classifications = classifications[['genre.id','genre.name','subGenre.id','subGenre.name']]

    # Updating index for future join
    classifications.index = events_json.index

    return classifications


classifications = process_classifications(events_df)
print(classifications.shape)
classifications.head()

(5, 4)


,genre.id,genre.name,subGenre.id,subGenre.name
68,KnvZfZ7vAv1,Hip-Hop/Rap,KZazBEonSMnZfZ7vkdA,Urban
162,KnvZfZ7vAeF,World,KZazBEonSMnZfZ7vFdJ,Latin
163,KnvZfZ7vAv1,Hip-Hop/Rap,KZazBEonSMnZfZ7vkdA,Urban
166,KnvZfZ7vAv1,Hip-Hop/Rap,KZazBEonSMnZfZ7vkdA,Urban
183,KnvZfZ7vAv6,Country,KZazBEonSMnZfZ7vAFa,Country


In [10]:
# Flattening price range JSON
def process_priceRange(events_json):

    # Create empty DF
    priceRange = pd.DataFrame()

    # Extracting price range information row by row
    for i in events_json.index:
        observation = json_normalize(events_json.loc[i, 'priceRanges'])
        priceRange = priceRange.append(observation)


    # Upating column names for clarity
    priceRange.rename(columns={'max':'price.max', 'min':'price.min'}, inplace=True)

    # Create attribute for max/min price differential
    priceRange['max_min.differential'] = priceRange['price.max']-priceRange['price.min']


    # Keeping valuable features
    priceRange = priceRange[['price.max','price.min','max_min.differential']]
    # Updating index for future join
    priceRange.index = events_json.index

    return priceRange


priceRange = process_priceRange(events_df) 
print(priceRange.shape)
priceRange.head()

(5, 3)


,price.max,price.min,max_min.differential
68,60.5,60.5,0.0
162,79.0,69.0,10.0
163,25.0,22.0,3.0
166,25.0,22.0,3.0
183,505.0,240.0,265.0


In [16]:
def process_page (events_json):

    # Extract all relvant attributes
    artists = process_artists(events_df)
    venues = process_venues(events_df)
    classifications = process_classifications(events_df)
    priceRange = process_priceRange(events_df) 
    id_name = events_json[['id','name']]
    dates = events_json[['dates.spanMultipleDays','dates.start.localDate','dates.start.localTime',
                       'dates.status.code','sales.public.startDateTime']]
    sales = events_json[['sales.presales']].notnull()
    promoter = events_json[['promoter.name']]
    
    # Concatenate all attributes
    attributes = pd.concat([id_name, artists, venues, classifications, priceRange, dates, sales, promoter], axis=1)

    # Remove cancelled events
    attributes = attributes[attributes['dates.status.code'] != 'cancelled']

    return attributes

attributes = process_page(events_df)
print(attributes.shape)
attributes.head(20)

(1, 30)


,id,name,primary.artistID,primary.artistName,artist.Count,multi.genres,multi.subGenres,artist.upcomingEvents.ticketmaster,artist.upcomingEvents.total,venue.id,...,price.max,price.min,max_min.differential,dates.spanMultipleDays,dates.start.localDate,dates.start.localTime,dates.status.code,sales.public.startDateTime,sales.presales,promoter.name
183,1ApZA8sGkd1Lokp,2018 B105 Country Megaticket,K8vZ917G0w0,Country Megaticket,8.0,0.0,0.0,230.0,248.0,KovZpZA6tdaA,...,505.0,240.0,265.0,True,2019-12-31,NaN,onsale,2018-01-26T15:00:00Z,True,LIVE NATION MUSIC


## Extracting All Data From Ticketmaster API (w/ Features Engineered)

In [21]:
# Setting API params (US based, music events only)
params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
          'countryCode':'US',                  # US based events only
          'segmentName':'Music',               # Music events only
          'sort':'date,desc',                  # Sorted by descending date
          'size':'200'}                        # API only allows 200 events pulled at a time

# Calling Ticketmaster API
response = requests.get('https://app.ticketmaster.com/discovery/v2/events.json', params=params)

# Converting response to JSON file
response_json = response.json()

In [22]:
# Convert JSON response to DataFrame
response_df = json_normalize(response_json)

page_count = response_df['page.totalPages'][0]

print ('Total Events: {}'.format(response_df['page.totalElements'][0]))
print ('Total Pages: {}'.format(response_df['page.totalPages'][0]))

Total Events: 32665
Total Pages: 164


In [43]:
# Create empty DF to store data
ticketmaster_data = pd.DataFrame()

# Looping through each page and appending to dataset
for page in range(1, page_count+1):
    
    # Set API params
    params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
              'countryCode':'US',                  # US based events only
              'segmentName':'Music',               # Music events only
              'sort':'date,desc',                  # Sorted by descending date
              'size':'200',                        # API only allows 200 events pulled at a time
              'page':str(page)                     # Set page number
             }
    
    # Call API
    response = requests.get('https://app.ticketmaster.com/discovery/v2/events.json', params=params)

    # Converting response to JSON file
    response_json = response.json()
    
    # Convert JSON response to DataFrame
    response_df = json_normalize(response_json)
    
    # Extract events data
    events_df = json_normalize(response_df['_embedded.events'][0])
    
    # Drop null values
    events_df.dropna(axis=0,
                     inplace=True,
                     subset=[['id',
                              'name',
                              '_embedded.attractions',
                              '_embedded.venues',
                              'classifications',
                              'priceRanges']])
    
    # Process attributes
    attributes = process_page(events_df)
    
    # Append results to DF
    ticketmaster_data = ticketmaster_data.append(attributes)
    
    # Keep track of progress
    print (page)

1
2
3
4


KeyError: '_embedded.events'

In [44]:
ticketmaster_data

,id,name,primary.artistID,primary.artistName,artist.Count,multi.genres,multi.subGenres,artist.upcomingEvents.ticketmaster,artist.upcomingEvents.total,venue.id,...,price.max,price.min,max_min.differential,dates.spanMultipleDays,dates.start.localDate,dates.start.localTime,dates.status.code,sales.public.startDateTime,sales.presales,promoter.name
120,vv170Z4kGkwtxKYm,Elton John: Farewell Yellow Brick Road,K8vZ9171G-7,Elton John,1.0,0.0,0.0,64.0,107.0,KovZpakTme,...,249.50,59.50,190.00,False,2019-09-11,20:00:00,onsale,2018-04-20T17:00:00Z,True,AEG LIVE
123,vv170Z4kGkwt-dYi,Elton John: Farewell Yellow Brick Road,K8vZ9171G-7,Elton John,1.0,0.0,0.0,64.0,107.0,KovZpakTme,...,249.50,59.50,190.00,False,2019-09-10,20:00:00,onsale,2018-04-20T17:00:00Z,True,AEG LIVE
133,k7vGF413EE7TS,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpakSbe,...,89.50,29.50,60.00,False,2019-08-24,19:30:00,onsale,2018-05-19T14:00:00Z,True,PROMOTED BY VENUE
144,vvG17Z41xkC39P,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpa2gne,...,694.50,25.00,669.50,False,2019-08-15,19:30:00,onsale,2018-05-19T14:00:00Z,True,AEG LIVE
147,1AvfZ4aGkRbhU11,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpaKuJe,...,695.00,25.50,669.50,False,2019-08-13,19:30:00,onsale,2018-05-19T14:00:00Z,True,THE MESSINA GROUP (TMG)
150,vvG1FZ413cifgU,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpZAE7vaA,...,699.00,29.50,669.50,False,2019-08-10,19:30:00,onsale,2018-05-19T14:00:00Z,True,PROMOTED BY VENUE
154,17AOvfG61CQ_Umn,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpZA6taeA,...,694.25,28.00,666.25,False,2019-08-06,19:30:00,onsale,2018-05-19T14:00:00Z,True,PROMOTED BY VENUE
157,vvG1OZ412Qn9Ze,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZ917A25V,...,699.00,24.50,674.50,False,2019-08-05,19:30:00,onsale,2018-05-19T14:00:00Z,True,AEG LIVE
161,G5viZ413aLNTZ,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpZA6taAA,...,85.00,25.80,59.20,False,2019-08-02,19:30:00,onsale,2018-05-19T15:00:00Z,True,AEG LIVE
163,vvG1zZ41GFyQgS,SHAWN MENDES: THE TOUR,K8vZ91739Nf,Shawn Mendes,1.0,0.0,0.0,41.0,73.0,KovZpa2Xke,...,85.50,25.50,60.00,False,2019-07-31,19:30:00,onsale,2018-05-19T14:00:00Z,True,AEG LIVE


## Adding Inventory Status

In [50]:
# Setting params
params = {'apikey':'4XrkbIMbr4gS3vG1qsFiZpDcXSbyAZni',
          'events':list(ticketmaster_data['id'])}

inventory = requests.get('https://app.ticketmaster.com/inventory-status/v1/availability', params=params)

inventory_json = inventory.json()

df = json_normalize(inventory_json)

df

,eventId,status
0,vvG17Z4GpqrfLs,TICKETS_NOT_AVAILABLE
1,vvG1YZ4GLeh4Fz,TICKETS_NOT_AVAILABLE
2,vvG17Z4Gpji0qN,TICKETS_NOT_AVAILABLE
3,17k8vfG6uQnRdqz,TICKETS_NOT_AVAILABLE
4,vvG1bZ4CUbRgZx,TICKETS_NOT_AVAILABLE
5,G5viZ42vTz37L,TICKETS_NOT_AVAILABLE
6,k7vGF4CmBAFig,TICKETS_NOT_AVAILABLE
7,vvG1fZ4Giq-y8y,TICKETS_NOT_AVAILABLE
8,17k8vfG6uQnmdqs,TICKETS_NOT_AVAILABLE
9,17FZvfG6CmT0bwg,TICKETS_NOT_AVAILABLE
